## 11-785 Project
### ChexNet

We followed the training strategy described in the official paper, and a ten crop method is adopted both in validation and test. Compared with the original CheXNet, the per-class AUROC of our reproduced model is almost the same. We have also proposed a slightly-improved model which achieves a mean AUROC of 0.847 (v.s. 0.841 of the original CheXNet).

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim    #optim.lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset
import torchvision #torchvision.datasets, torchvision.models, torchvision.transforms
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau #Reduce learning rate when a metric has stopped improving

import numpy as np


import matplotlib.pyplot as plt
import datetime, os, copy
from pathlib import Path
from collections import OrderedDict, namedtuple

# 'PIL' is the Python Imaging Library, 
# 'Image' module provides a class to represent a PIL image. 
## it provides factory functions, like load images from files, and to create new images.
from PIL import Image


#Used to compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
from sklearn.metrics.ranking import roc_auc_score

from sklearn.preprocessing import normalize


In [18]:
print(torch.__version__)

0.4.0


### Auxiliary Functions

In [19]:
def load_raw(path, name):
    file = Path(path) / name
    if file.exists():
        return np.load(file,encoding='bytes')
    else:
        raise Exception("File not found chutia!!")

In [20]:
def to_tensor(numpy_array):
    # Numpy array -> Tensor
    return torch.from_numpy(numpy_array)


def to_variable(tensor):
    # Tensor -> Variable (on GPU if possible)
    if torch.cuda.is_available():
        # Tensor -> GPU Tensor
        tensor = tensor.cuda()
    return torch.autograd.Variable(tensor)

In [21]:
#taken from github of arnoweng/CheXNet
def compute_AUCs(y_hat, y_true, args):
    AUROCs = []
    
    y_hat = normalize(y_hat, axis=1, norm='l1')
    print (y_true.shape)
    print (y_hat.shape)
    #print (roc_auc_score(y_true, y_hat))
    for i in range(args.n_classes):
        AUROCs.append(roc_auc_score(y_true[:, i], y_hat[:, i]))
        
    AUROC_avg = np.array(AUROCs).mean()
    print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
    for i in range(args.n_classes):
        print('The AUROC of {} is {}'.format(args.disease_categories[i], AUROCs[i]))

### Pytorch Model  

In [22]:
class DenseNet121(nn.Module):
    #!!!def __init__(self, classCount, isTrained):
    def __init__(self, args):
        #!!!super(DenseNet121, self).__init__()
        super().__init__()
        self.densenet121 = None
        
        if args.backprop_pretained:
            #Fixed Feature Extractor
            #freeze the weights for all of the network except that of the final fully connected layer. 
            
            self.densenet121 = torchvision.models.densenet121(pretrained=True)
            for param in self.densenet121.parameters():
                param.requires_grad = False
            
            #parameters = filter(lambda p: p.requires_grad, self.densenet121.parameters())
            #for param in parameters:
            #    param.requires_grad = False
                
        else:
            #Finetuning 
            #initialize the network with a pretrained networt. Rest of the training looks as usual.
            self.densenet121 = torchvision.models.densenet121(pretrained=True)
            
        
        # Parameters of newly constructed modules have requires_grad=True by default
        #fc -> contains the last layer of network (only for resnet)
        #classifier -> -> contains the last layer of network (only for densenet)
        
        ##in RESNET last layer is from 2048 to 1000
        #num_features = dcnn.fc.in_features 
        
        ##in DENSENET last layer is from 1024 to 1000
        num_features = self.densenet121.classifier.in_features
        
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_features, args.n_classes),
            nn.Sigmoid()
        )
        
        
    #each image should be size 224x224 as original paper of Densenet states
    def forward(self, input_val):
        y = self.densenet121(input_val)
        return y
    
                    
    def initialize_weigths(self, args):
        pass


### Pytorch Dataset

All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224. The images have to be loaded in to a range of [0, 1] and then normalized using 
`normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])`

In [23]:
#class ChestXray(torch.utils.data.Dataset):
class ChestXray (torch.utils.data.TensorDataset):
    def __init__(self, args, dataset_list_file, path, is_val=False):
        
        
        normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                         [0.229, 0.224, 0.225])
        
        #used in variation of CheXnet of Weng,Zhuang,Tian
        self.transform1 = transforms.Compose([
                                transforms.Resize(256),
                                transforms.TenCrop(224), #return  list of 10 images 
                                transforms.Lambda (lambda crops: torch.stack([transforms.ToTensor()(x) for x in crops])),
                                transforms.Lambda(lambda crops: torch.stack([normalize(x) for x in crops]))
                            ])
        
        #used in original paper of CheXnet of Rajpurkar,Irvin,Zhu,Ng
        self.transform2 = transforms.Compose([
                                transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                normalize
                            ])
        self.is_val = is_val
            
        image_names = []
        labels = []
        tmp = "all_images"
        with open(os.path.join(path, dataset_list_file), "r") as file:
            for line in file:
                items = line.split(',')
                label = [int(i) for i in items[1:]]
                labels.append(label)

                image_filename = items[0]
                image_name = os.path.join(path,tmp, image_filename)
                image_names.append(image_name)
                
        self.image_names = image_names
        self.labels = labels
        print ("There are {} images in the Images Dataset".format(len(self.image_names)))
        
    def __getitem__(self, index):

        
        image_filename = self.image_names[index]
        image = Image.open(image_filename).convert('RGB')
        label = torch.FloatTensor(self.labels[index])
        if args.transform == "transform_1":
            if self.is_val == False:
                image = self.transform1(image)
            else:
                image = self.transform2(image)
        elif args.transform == "transform_2":
            image = self.transform2(image)
        return image, label

    def __len__(self):
        return len(self.image_names)
    


### Validation Routine

In [24]:
#routine to compute loss based on Validation dataset
def validate_routine(model, args, val_datalist,path):
    
    model.eval() #DO NOT FORGET to do evaluation
    
    loss = nn.BCELoss()
    
    dataset = ChestXray(args, val_datalist, path, is_val=True)
    data_loader = torch.utils.data.DataLoader(
                    dataset, batch_size=args.batch_size, shuffle=False,
                    num_workers=args.num_workers, pin_memory=args.pin_memory)
    
    losses = []
    for i,(input_val,labels) in enumerate(data_loader): 
            
        #forward pass
        #print ("val batch processing: ",i)
        prediction = model(to_variable(input_val))

        #print("Finished forward pass")
        val_loss = loss(prediction, to_variable(labels))
        losses.append(val_loss.data.cpu().numpy())
            
    return losses

### Training Routine

In [35]:
def training_routine(args, path, train_datalist, val_datalist):
    
    #open files and load content
    
    # Create the network
    if args.load_disk_model==True:
        load_model = torch.load(args.load_checkpoint_filename, map_location=lambda storage, loc: storage)
        #load_model = torch.load('ChexNet_model2.pytorch')
        model = DenseNet121(args)
        new_state_dict = OrderedDict()
        for k, v in load_model.items():
            #print (k)
            name = k[7:] # remove `module.`
            new_state_dict[name] = v
        # load params
        model.load_state_dict(new_state_dict)
    elif args.load_dis_model == False:
        model = DenseNet121(args)  
    
    #Initialize weitgths
    #my_model.initialize_weigths()
    
    
    #Choose the loss function / optimizer
    loss = nn.BCELoss(size_average = True)
    
    #choose optimizer
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                             lr=args.learn_rate,
                             weight_decay=args.l2)
    scheduler = ReduceLROnPlateau(optim, factor = 0.1, patience = 10, mode = 'min')
                         
    print ("Created Neural Network arquitecture")
    
    if torch.cuda.is_available():
        # Move the network and the optimizer to the GPU
        print ("Moving to GPU")
        model = model.cuda()
        model = torch.nn.DataParallel(model).cuda()
        loss = loss.cuda()
    
    dataset = ChestXray(args, train_datalist, path, is_val=False)
    
    
    data_loader = torch.utils.data.DataLoader(
                    dataset, batch_size=args.batch_size, shuffle=True,
                    num_workers=args.num_workers, pin_memory=args.pin_memory)
    
    print ("Created data objects")
    
    losses= []
    for epoch in range(args.epochs): 
        model.train()
        t0 = datetime.datetime.now()
        losses = []
        for i,(input_val,labels) in enumerate(data_loader): 

            #if transform_1 using, we got a vector of 5 DIM
            #and we only need a 4 DIM
            if args.transform == 'transform_1':  
                bs, n_crops, c, h, w = input_val.size() #e.g(4,10,3,224,244)
                input_val = input_val.view(-1, c, h, w) #e.g(40,3,224,224)
                bs, n_classes = labels.size() #e.g(4,14)
                labels = labels.unsqueeze(2).repeat(1, n_crops, 1 )
                labels = labels.contiguous().view(bs*n_crops, n_classes)
                
            prediction = model(to_variable(input_val))

            #print("Finished forward pass")
            #print (prediction.shape)
            
            train_loss = loss(prediction, to_variable(labels))
            optim.zero_grad()# Reset the gradients NEVER FORGET THIS
            train_loss.backward()
            optim.step() # Update the network
            
            losses.append(train_loss.data.cpu().numpy())

            
            if i % 200 == 0:
                print('Minibatch ',i,train_loss.data.cpu().numpy())
            if i % 800 == 0:
                val_loss = validate_routine(model, args, val_datalist, path)
                scheduler.step(np.asscalar(np.mean(val_loss)))
                model.train()
            
            
        print ('EPOCH', end='\t')
        print ("Epoch {} Train Loss: {:.4f}".format(epoch, np.asscalar(np.mean(losses))), end='\t')
        print ("Epoch {} Validation Loss: {:.4f}".format(epoch, np.asscalar(np.mean(val_loss))), end='\t')
        print ("Epoch Time: {}".format(datetime.datetime.now()-t0))
        
        torch.save(model.state_dict(), args.save_checkpoint_filename)
        
    return model,losses


### Define parameters and dataset and RUN

In [26]:
disease_categories = {'Atelectasis': 0, 'Cardiomegaly': 1, 'Effusion': 2,
                          'Infiltration': 3, 'Mass': 4, 'Nodule': 5, 'Pneumonia': 6,
                          'Pneumothorax': 7, 'Consolidation': 8, 'Edema': 9,
                          'Emphysema': 10, 'Fibrosis': 11, 'Pleural_Thickening': 12, 'Hernia': 13,
                          }

disease_categories2 = {val:key for (key, val) in disease_categories.items()}

col_nanes = ['FileName', 'Atelectasis', 'Cardiomegaly', 'Effusion', 
             'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 
             'Pneumothorax', 'Consolidation', 'Edema', 
             'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia'
            ]


In [33]:
#in GPU
args = {'load_disk_model':True, 'load_checkpoint_filename':'ChexNet_model4.pytorch',
        'save_checkpoint_filename':'ChexNet_model4.pytorch',
        'backprop_pretained':True, #if False do finetuning if True just Fixed Feature Extractor
        'n_classes':14, 'transform':'transform_2', 'disease_categories':disease_categories2,
        'num_workers':16, 'pin_memory':True, #used for DataLoader
        'batch_size':16, 'epochs':8,
        'learn_rate':0.000001, 'l2':0.1e-08, #used in optimization 
       }

#in CPU
#args['num_workers'] = 16
#args['pin_memory'] = True

args = namedtuple('args', args.keys())(**args)

path = "dataset"
train = "train_set.csv" 
val = "val_set.csv"
test = "test_set.csv"

In [36]:
a = datetime.datetime.now()
print (a)
print ("Start Training")
model,losses = training_routine(args, path, train, val)
print ("I am done training")
b = datetime.datetime.now()
print (b)
print (b-a)


2018-05-03 16:40:51.038897
Start Training


/home/ubuntu/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


Created Neural Network arquitecture
Moving to GPU
There are 78826 images in the Images Dataset
Created data objects
Minibatch  0 0.20403613150119781
There are 11140 images in the Images Dataset


Process Process-377:
Process Process-389:
Process Process-370:
Process Process-390:
Process Process-388:
Process Process-386:
Process Process-391:
Process Process-382:
Process Process-371:
Process Process-387:
Process Process-369:
Process Process-383:
Process Process-372:
Process Process-376:
Process Process-384:
Process Process-381:
Process Process-378:
Process Process-373:
Traceback (most recent call last):
Process Process-380:
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process Process-385:
Process Process-375:
Process Process-374:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-379:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most rec

  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-23-4f22b2c75cfd>", line 47, in __getitem__
    image = Image.open(image_filename).convert('RGB')
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/an

  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "<ipython-input-23-4f22b2c75cfd>", line 47, in __getitem__
    image = Image.open(image_filename).convert('RGB')
  File "<ipython-input-23-4f22b2c75cfd>", line 47, in __getitem__
    image = Image.open(image_filename).convert('RGB')
KeyboardInterrupt
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/functional.py", line 331, in resized_crop
    img = resize(img, size, interpolation)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_

  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py", line 860, in convert
    self.load()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py", line 860, in convert
    self.load()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py", line 1712, in resize
    return self._new(self.im.resize(size, resample))
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/ImageFile.py", l

KeyboardInterrupt: 

### Predict

In [ ]:
use_weng = False
if use_weng:
    load_model = torch.load('transfer_model/model.pth.tar', map_location=lambda storage, loc: storage)

    model = DenseNet121(args)
    
    new_state_dict = OrderedDict()
    for k, v in load_model.items():
        if k == 'state_dict':
            for k2,v in v.items():
                #print (k2)
                name = k2[7:] # remove `module.`
                new_state_dict[name] = v
    # load params
    model.load_state_dict(new_state_dict)
    print ("Model Loaded, using already trained model")



In [37]:
use_from_disk = True
if use_from_disk:
    load_model = torch.load(args.load_checkpoint_filename, map_location=lambda storage, loc: storage)
    #load_model = torch.load('ChexNet_model2.pytorch')
    model = DenseNet121(args)
    new_state_dict = OrderedDict()
    for k, v in load_model.items():
        #print (k)
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    # load params
    model.load_state_dict(new_state_dict)
    #model.load_state_dict(load_model)
    if torch.cuda.is_available():
        # Move the network and the optimizer to the GPU
        print ("Moving to GPU")
        model = model.cuda()
        model = torch.nn.DataParallel(model).cuda()
    #print(model) 


t0 = datetime.datetime.now()

model.eval() #DO NOT FORGET to do evaluation


dataset = ChestXray(args, test, path, is_val=True)
data_loader = torch.utils.data.DataLoader(
                dataset, batch_size=ags.batch_size, shuffle=False,
                num_workers=args.num_workers, pin_memory=args.pin_memory)


y_hat = []
y_true = []
for i,(input_val,labels) in enumerate(data_loader): 

    #forward pass
    if i % 50 == 0:
        print ("val batch processing: ",i)
                
    prediction = model(to_variable(input_val))
    #print ("Donde Forward Pass")
    y_true.append(labels.cpu().numpy().astype(int))
    y_hat.append(prediction.data.cpu().numpy())
    

y_hat = np.vstack(y_hat)
y_true = np.vstack(y_true)
compute_AUCs(y_hat, y_true, args)
print ("Predict Time: {}".format(datetime.datetime.now()-t0))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


Moving to GPU
There are 22152 images in the Images Dataset


NameError: name 'ags' is not defined

#### CHINESE MODEL TO BEAT

* The average AUROC is 0.859
* The AUROC of Atelectasis is 0.9205465587044535
* The AUROC of Cardiomegaly is 0.9374262101534828
* The AUROC of Effusion is 0.9317375886524822
* The AUROC of Infiltration is 0.49387630128597676
* The AUROC of Mass is 0.9601226993865031
* The AUROC of Nodule is 0.8991769547325104
* The AUROC of Pneumonia is 0.6707317073170731
* The AUROC of Pneumothorax is 0.9375
* The AUROC of Consolidation is 0.73375
* The AUROC of Edema is 0.9878048780487805
* The AUROC of Emphysema is 0.9503105590062111
* The AUROC of Fibrosis is 0.7670807453416149
* The AUROC of Pleural_Thickening is 0.834355828220859
* The AUROC of Hernia is 1.0

#### Model without training
The average AUROC is 0.439
* The AUROC of Atelectasis is 0.4600202429149797
* The AUROC of Cardiomegaly is 0.49350649350649356
* The AUROC of Effusion is 0.5774231678486997
* The AUROC of Infiltration is 0.5272504592774034
* The AUROC of Mass is 0.3098159509202454
* The AUROC of Nodule is 0.3786008230452675
* The AUROC of Pneumonia is 0.426829268292683
* The AUROC of Pneumothorax is 0.23750000000000004
* The AUROC of Consolidation is 0.5125000000000001
* The AUROC of Edema is 0.5914634146341464
* The AUROC of Emphysema is 0.3944099378881988
* The AUROC of Fibrosis is 0.14440993788819875
* The AUROC of Pleural_Thickening is 0.7668711656441718
* The AUROC of Hernia is 0.3292682926829268

#### Model trained 3 epochs, transform1
* The average AUROC is 0.522
* The AUROC of Atelectasis is 0.5600628216848379
* The AUROC of Cardiomegaly is 0.4994946999599024
* The AUROC of Effusion is 0.5750399977244025
* The AUROC of Infiltration is 0.5460339649011673
* The AUROC of Mass is 0.5018732586055604
* The AUROC of Nodule is 0.5095718525039765
* The AUROC of Pneumonia is 0.4791604866435438
* The AUROC of Pneumothorax is 0.5447996763720493
* The AUROC of Consolidation is 0.5780586014503319
* The AUROC of Edema is 0.5659363209503817
* The AUROC of Emphysema is 0.44840684512882306
* The AUROC of Fibrosis is 0.5362195673894978
* The AUROC of Pleural_Thickening is 0.5302204565319748
* The AUROC of Hernia is 0.43561190809913514
Predict Time: 0:03:35.336959

#### Model Trained 15 epochs, transform1

* The average AUROC is 0.626
* The AUROC of Atelectasis is 0.6099339225725543
* The AUROC of Cardiomegaly is 0.6010881987356465
* The AUROC of Effusion is 0.7138617014446589
* The AUROC of Infiltration is 0.5488878033845505
* The AUROC of Mass is 0.5329107421959458
* The AUROC of Nodule is 0.5597421920386231
* The AUROC of Pneumonia is 0.5141474040099794
* The AUROC of Pneumothorax is 0.68892335847419
* The AUROC of Consolidation is 0.6874902348764493
* The AUROC of Edema is 0.789283092744088
* The AUROC of Emphysema is 0.6213053225614307
* The AUROC of Fibrosis is 0.6504512954572436
* The AUROC of Pleural_Thickening is 0.5792289384426821
* The AUROC of Hernia is 0.663563257739683

### Model Trained with 15 epochs and scheduler, transform1
* The average AUROC is 0.679
* The AUROC of Atelectasis is 0.6406096981917049
* The AUROC of Cardiomegaly is 0.6800817382395566
* The AUROC of Effusion is 0.7288424773864505
* The AUROC of Infiltration is 0.5384996004973043
* The AUROC of Mass is 0.5753880650294986
* The AUROC of Nodule is 0.6035420785256662
* The AUROC of Pneumonia is 0.5957072608616284
* The AUROC of Pneumothorax is 0.7298815534886969
* The AUROC of Consolidation is 0.7119627809094822
* The AUROC of Edema is 0.822888148544922
* The AUROC of Emphysema is 0.7044018976469857
* The AUROC of Fibrosis is 0.7071243037968423
* The AUROC of Pleural_Thickening is 0.6461906776431365
* The AUROC of Hernia is 0.8141235894819112


### Model3 Trained with 5 epochs and scheduler, transform2, doing finetuning

The average AUROC is 0.788
* The AUROC of Atelectasis is 0.7489327514696104
* The AUROC of Cardiomegaly is 0.8624223503416083
* The AUROC of Effusion is 0.8481957794094263
* The AUROC of Infiltration is 0.5208833490502071
* The AUROC of Mass is 0.8112666095907779
* The AUROC of Nodule is 0.7241843444330321
* The AUROC of Pneumonia is 0.707121858646708
* The AUROC of Pneumothorax is 0.8551896787260633
* The AUROC of Consolidation is 0.7767152224992842
* The AUROC of Edema is 0.8971054331868876
*  The AUROC of Emphysema is 0.8611092544828925
* The AUROC of Fibrosis is 0.7887263006509438
* The AUROC of Pleural_Thickening is 0.7237187698617101
* The AUROC of Hernia is 0.9127174313230288
Predict Time: 0:03:04.180466

### Model4 Trained over "model3" with 5 epochs and scheduler, transform2, doing finetuning

The average AUROC is 0.790
* The AUROC of Atelectasis is 0.7473135574453651
* The AUROC of Cardiomegaly is 0.872459265342519
* The AUROC of Effusion is 0.8441989498985264
* The AUROC of Infiltration is 0.5442143078958999
* The AUROC of Mass is 0.8140306853786841
* The AUROC of Nodule is 0.7218175283825453
* The AUROC of Pneumonia is 0.7056280044420105
* The AUROC of Pneumothorax is 0.8604332962402138
* The AUROC of Consolidation is 0.7728041755325066
* The AUROC of Edema is 0.8994701265347935
* The AUROC of Emphysema is 0.8670938030633245
* The AUROC of Fibrosis is 0.8028644493688417
* The AUROC of Pleural_Thickening is 0.7183141564743182
* The AUROC of Hernia is 0.8934911089724129
Predict Time: 0:03:00.684701